# Análisis de Mortalidad Fetal en Guatemala (2012-2022) mediante Clustering K-Means

Daniel Chet - 231177, Dulce Ambrosio - 231143

---

### Situación Problemática

Guatemala registra una de las tasas de mortalidad fetal más altas de Latinoamérica. El presente proyecto busca analizar los datos oficiales de **Defunciones Fetales y Nacimientos (2012-2022)** publicados por el Instituto Nacional de Estadística (INE) para identificar patrones y perfiles de riesgo mediante técnicas de análisis exploratorio y minería de datos (K-Means). El objetivo es descubrir si existen segmentos diferenciados de la población materna que presenten mayor vulnerabilidad a la muerte fetal, considerando variables como edad, etnia, escolaridad, asistencia recibida y semanas de gestación.